## Fund Separation Theorem and Capital Market Line

In the presence of a risk-free asset, the efficient frontier becomes a straight line which is called **capital market line**. This line represents the tangency portfolio, which is the porfolio that maximizes the Sharpe ratio (reward-per-risk ratio) given a risk-free asset. In other words, the capital market line is the line tangent to the efficient frontier with maximum slope that also crosses the y-axis (return axis) at the risk-free rate of return. If you have a look at the equation of the Sharpe ratio, you will notice that it gives the slope of any straight line in the volatility-return plane that has a value of the risk-free rate at the origin. If we are considering N assets, the Sharpe ratio is given by:


$$ SR_{portfolio} = \frac{\mu_{portfolio} - r_{f}}{\sigma_{portfolio}}  = \frac{\sum_{i=1}^{N} w_{i}R_{i}  - r_{f}}{\sqrt{\sum_{i=1}^{N}\sum_{j=1}^{N} w_{i}\sigma_{i,j} w_{j}}} $$

where $r_{f}$ is the risk-free rate (bear in mind that it is outside the summatory). In order to find the capital market line, we must find the weights $w$ that maximize the previous equation.

### Locating Max Sharpe Ratio Portfolio

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import edhec_risk_kit as erk
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [3]:
ind = erk.get_ind_returns()
er = erk.annualized_rets(ind["1996":"2000"],12)
cov = ind["1996":"2000"].cov()

In order to find the Max Sharpe ratop, we have to define a function that is pretty similar to the one we built for minimizing the volatility. However, the function that we can find below is a bit larger, as it introduces more elements and data to be plotted. In any case, the essence of it is more or less the same.

In [4]:
def maximize_sr(riskfree_rate,er,cov,n_points=None, plot=False):
    """
    Given some constraints and an initial guess, it returns a DataFrame with the weights
    that produce the portfolio with maximum Sharpe ratio, the return and volatility
    that produce that Sharpe ratio, and the maximized Sharpe ratio. If plot == True,
    it plots the efficient frontier, the Capital Market line and the point of tangency
    of both. In this case, it is also require to choose the number of points for the 
    efficient frontier
    """
    
    n = er.shape[0] #number of assets
    init_guess = np.repeat(1/n,n) # same weights for everyone
    
    #We need some constraints
    
    bounds = ((0.0,1.0),)*n # w create n tuples with bound from 0 to 1
    
    weights_sum_to_1 = {
        'type': 'eq',
        'fun': lambda weights: np.sum(weights) - 1
    }
    
    # Please, have a look at minimize_vol for more explanation. The difference is that instead of maximizing
    # Sharpe ratio, we minimize the negative Sharpe ratio, so we obtain the maximized Sharpe ratio, but with 
    # a negative sign.
    
    def neg_sharpe_ratio(weights,riskfree_rate,er,cov):
        """
        Returns the negative of Sharpe ratio.
        """
        r = erk.portfolio_return(weights,er)
        vol = erk.portfolio_vol(weights,cov)
        return - (r-riskfree_rate)/vol
    
    minimized_weights = minimize(neg_sharpe_ratio,init_guess, args=(riskfree_rate,er,cov,), method = "SLSQP", options={"disp": False}, constraints=(weights_sum_to_1), bounds=bounds)

    #We create the DataFrame with minimized weights, vol, return and max Sharpe ratio
    
    max_sharpe_ratio = -neg_sharpe_ratio(minimized_weights.x,riskfree_rate,er,cov)
    
    minimized_weights_dataframe = pd.DataFrame(np.reshape(minimized_weights.x,(1,minimized_weights.x.shape[0])), columns = er.index)
    ret_vol_max_sr_dataframe = pd.DataFrame({"Returns": [erk.portfolio_return(minimized_weights.x,er)], 
                       "Volatility":[erk.portfolio_vol(minimized_weights.x,cov)],
                        "Maximized Sharpe Ratio": [max_sharpe_ratio]
                                     })
    df = pd.concat([ret_vol_max_sr_dataframe,minimized_weights_dataframe], axis=1)
    
    if plot == False: #returns only dataframe
        
        return df
    
    elif plot == True: #returns DataFrame and also plots CML, EF and tangency point.
        
        ef_data = erk.ef_multi(n_points,er,cov,plot=True) # we have to set the number of points for the frontier
        
        def cml(x,riskfree_rate,max_sharpe_ratio): #we create the function for the Capital Market Line
            
            return riskfree_rate + max_sharpe_ratio*x
        
        x = np.linspace(0, ret_vol_max_sr_dataframe["Volatility"],n_points) #we choose the points the line will have
        
        #Plotting CML and Tangency Point
        plt.plot(x,cml(x,riskfree_rate,max_sharpe_ratio), label="Capital Market Line",linestyle="dashed")
        plt.scatter(ret_vol_max_sr_dataframe["Volatility"],ret_vol_max_sr_dataframe["Returns"], label = "Tangency Point", s=8, c="green",marker='o' )
        plt.legend()
        plt.xlim(0,ef_data["Volatility"].max())
        plt.title("Efficient Frontier and Capital Market Line")
        
        return df
    
    
    

In [6]:
#erk.ef_complete(0.10,er,cov,show_sr=True)